In [38]:
import random
from bs4 import BeautifulSoup
import requests
import pandas as pd

header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_name_dict = {'ATL': 'Atlanta',
                        'WSN': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinatti',
                        'PIT': 'Pittsburg',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SFG': 'San Francisco',
                        'COL': 'Colorado',
                        'SDP': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TBR': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KCR': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}

In [5]:
class Player:
    """Class that holds information for a player.
    
    Attributes:
        team: team name of player
        name: player name
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, singles, doubles, triples, home_runs, walks):
        """Initializes values for this class"""
        
        self.team = team
        self.name = name
        self.singles = singles
        self.doubles = singles + doubles
        self.triples = singles + doubles + triples
        self.home_runs = singles + doubles + triples + home_runs
        self.walks = singles + doubles + triples + home_runs + walks
        self.cum_singles = 0
        self.cum_doubles = 0
        self.cum_triples = 0
        self.cum_home_runs = 0
        self.cum_walks = 0
        self.cum_pa = 0
        self.cum_runs = 0
        self.cum_rbi = 0
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if(self.cum_pa-self.cum_walks>0):
            return(float((self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs)/(self.cum_pa-self.cum_walks)))
        else:
            return 0    
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form"""
        if(self.cum_pa>0):
            return((self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs+self.cum_walks)/(self.cum_pa))
        else:
            return 0
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form"""
        if(self.cum_pa-self.cum_walks>0):
            return((self.cum_singles+2*self.cum_doubles+3*self.cum_triples+4*self.cum_home_runs)/(self.cum_pa-self.cum_walks))
        else:
            return 0
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form"""
        
        return(self.slg() + self.obp())
    def print_stats(self):
        """Prints some stats of a player"""

        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str(self.cum_home_runs))

In [6]:
class Team:
    """Class that holds information for a team.
    
    Attributes:
        name: team name
        p1-p9: instances of player class
        roster: batting lineup of team in list form
        index: position in batting order of current batter
        runners: list representing whether a runner is on each base
        runs: number of runs fr a team in a game
        wins: number of wins for a team
        losses: number of losses for a team
        cumulative runs: total runs for a team over a span of games
    """
    def __init__(self, name, p1, p2, p3, p4, p5, p6, p7, p8, p9):
        """Initializes values for this class"""
        self.name = name
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.p4 = p4
        self.p5 = p5
        self.p6 = p6
        self.p7 = p7
        self.p8 = p8
        self.p9 = p9
        self.roster = [p1, p2, p3, p4, p5, p6, p7, p8, p9]
        self.index = 0
        self.runners = [None, None, None, 0]    # first base-hom plate. 0 for base empty, 1 for runner on
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started"""
        self.index = 0
        self.runners = [None, None, None, 0]
        self.runs = 0
    def reset_runners(self):
        """Resets baserunners after each half inning"""
        self.runners = [None, None, None, 0]

In [7]:
class Game:
    """Class that holds information for a Game.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.home_team = home_team
        self.away_team = away_team
    def get_result(self, player, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            player: instance of a player class
            num: random number between 0 and 1
        Returns:
            str: refers to result based on random number (ie "walk")
        """
        player.cum_pa+=1
        
        if(num<player.singles):
            player.cum_singles+=1
            return "single"
        elif(num<player.doubles):
            player.cum_doubles+=1
            return "double"
        elif(num<player.triples):
            player.cum_triples+=1
            return "triple"
        elif(num<player.home_runs):
            player.cum_home_runs+=1
            return "home_run"
        elif(num<player.walks):
            player.cum_walks+=1
            return "walk"
        else:
            return "out"  
    def move_runners(self, team, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            result: str of reult for batter (ie "walk")
        """
        if(result=="single"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = team.roster[team.index]

        elif(result=="double"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[0]
            team.runners[1] = team.roster[team.index]
            team.runners[0] = None

        elif(result=="triple"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.roster[team.index]
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="home_run"):
            team.roster[team.index].cum_runs+=1
            team.roster[team.index].cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.roster[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="walk"):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            team.runners[0] = team.roster[team.index]
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
                    if(temp_2!=None):
                        team.runners[3]+=1

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0     
    def print_result(self):
        """Printed score of a game""" 
        print("final score: " + away_team.name + " " + str(away_team.runs) + " " + home_team.name + " " + str(home_team.runs))     
    def print_record(self):
        """Printed score of a game""" 
        print(away_team.name + ": " + str(away_team.wins) + "-" + str(away_team.losses))
        print(home_team.name + ": " + str(home_team.wins) + "-" + str(home_team.losses))     
    def play(self):
        """A game between two instances of the team class is played""" 
        inning = 1
        while(inning<10 or away_team.runs==home_team.runs):
            for side in [away_team, home_team]:
                outs = 0

                while(outs<3):
                    num = random.uniform(0,1)
                    result = self.get_result(side.roster[side.index], num)
                    if(result=="out"):
                        outs+=1
                    else:
                        self.move_runners(side, result)
                        
                    side.index+=1
                    side.index = side.index%8
                    
                side.reset_runners()
            inning+=1
        
        if(away_team.runs>home_team.runs):
            away_team.wins+=1
            home_team.losses+=1
        elif(away_team.runs<home_team.runs):
            away_team.losses+=1
            home_team.wins+=1
        
        away_team.cum_runs+=away_team.runs
        away_team.restart()
        home_team.cum_runs+=home_team.runs
        home_team.restart()              

In [8]:
class Season(Game):
    """Class that holds information for a Season of games.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.home_team = home_team
        self.away_team = away_team
    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(away_team, home_team)
            g.play() 

In [9]:
a1 = Player("Away", "1", .170, .080, .010, .030, .050)
a2 = Player("Away", "2", .170, .080, .010, .030, .050)
a3 = Player("Away", "3", .170, .080, .010, .030, .050)
a4 = Player("Away", "4", .170, .080, .010, .030, .050)
a5 = Player("Away", "5", .170, .080, .010, .030, .050)
a6 = Player("Away", "6", .170, .080, .010, .030, .050)
a7 = Player("Away", "7", .170, .080, .010, .030, .050)
a8 = Player("Away", "8", .170, .080, .010, .030, .050)
a9 = Player("Away", "9", .170, .080, .010, .030, .050)
h1 = Player("Home", "1", .170, .080, .010, .030, .050)
h2 = Player("Home", "2", .170, .080, .010, .030, .050)
h3 = Player("Home", "3", .170, .080, .010, .030, .050)
h4 = Player("Home", "4", .170, .080, .010, .030, .050)
h5 = Player("Home", "5", .170, .080, .010, .030, .050)
h6 = Player("Home", "6", .170, .080, .010, .030, .050)
h7 = Player("Home", "7", .170, .080, .010, .030, .050)
h8 = Player("Home", "8", .170, .080, .010, .030, .050)
h9 = Player("Home", "9", .170, .080, .010, .030, .050)
away_team = Team("Away", a1, a2, a3, a4, a5, a6, a7, a8, a9)
home_team = Team("Home", h1, h2, h3, h4, h5, h6, h7, h8, h9)

s = Season(away_team, home_team)
s.simulate_season(1000)
s.print_record()
a1.print_stats()

Away: 513-487
Home: 487-513
Avg: 0.308
OBP: 0.345
SLG: 0.507
OPS: 0.852
HR: 169


In [45]:
headers = {'User-Agent': header_name}
teams_dict = {}

for k, v in acronym_to_name_dict.items():
    URL = 'https://www.baseball-reference.com/teams/' + key + '/2019.shtml#all_team_batting'
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    df_columns = []
    
    for table in soup.findAll('table', attrs={'id': 'team_batting'}):
        for labels in table.findAll('thead'):
            for header_value in labels.findAll('th'):
                df_columns.append(header_value.get_text())

            df = pd.DataFrame(columns=df_columns)

        for player in table.findAll('tbody'):
            for row in player.findAll('tr'):
                if(row.attrs and row['class'][0]=='thead'):
                    continue

                player_values = []
                player_values.append(row.th.get_text())

                for column_value in row.findAll('td'):
                    player_values.append(column_value.get_text())
                df = df.append(pd.Series(player_values, index=df.columns),
                                                      ignore_index=True)
    teams_dict.update({v: df})
        
    print(teams_dict)

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

...

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 

{'Atlanta':     Rk Pos             Name Age    G   PA   AB   R    H  2B  ...   OBP   SLG  \
0    1   C    Omar Narvaez*  27  132  482  428  63  119  12  ...  .353  .460   
1    2  1B      Austin Nola  29   79  267  238  37   64  12  ...  .342  .454   
2    3  2B      Dee Gordon*  31  117  421  393  36  108  12  ...  .304  .359   
3    4  SS   J.P. Crawford*  24   93  396  345  43   78  21  ...  .313  .371   
4    5  3B     Kyle Seager*  31  106  443  393  55   94  19  ...  .321  .468   
..  ..  ..              ...  ..  ...  ...  ...  ..  ...  ..  ...   ...   ...   
62  63   P  Anthony Swarzak  33    0    0    0   0    0   0  ...               
63  64   P  Brandon Brennan  27    3    0    0   0    0   0  ...               
64  65   P      Ryan Garton  29    0    0    0   0    0   0  ...               
65  66   P       Art Warren  26    0    0    0   0    0   0  ...               
66  67   P      David McKay  24    0    0    0   0    0   0  ...               

     OPS OPS+   TB GDP HBP 